
# 7. Modelo CAPM

In [9]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

RISKY_ASSET = "AMZN"
MARKET_BENCHMARK = "^GSPC"
START_DATE = "2020-01-01"
END_DATE = "2024-12-31"

In [10]:
df = yf.download([RISKY_ASSET, MARKET_BENCHMARK],
                 start=START_DATE,
                 end=END_DATE,
                 auto_adjust=True,
                 multi_level_index=False)

[*********************100%***********************]  2 of 2 completed


In [11]:
X = (
    df["Close"]
    .rename(columns={RISKY_ASSET: "asset",
                     MARKET_BENCHMARK: "market"})
    .resample("ME")
    .last()
    .pct_change()
    .dropna()
)
X

Ticker,asset,market
Date,,
2020-02-29,-0.062214,-0.084110
2020-03-31,0.035021,-0.125119
2020-04-30,0.268900,0.126844
2020-05-31,-0.012785,0.045282
2020-06-30,0.129567,0.018388
2020-07-31,0.147114,0.055101
2020-08-31,0.090461,0.070065
2020-09-30,-0.087579,-0.039228
2020-10-31,-0.035754,-0.027666


In [12]:
covariance = X.cov().iloc[0,1]
benchmark_variance = X.market.var()
beta = covariance / benchmark_variance
beta

np.float64(1.1533821866569285)

In [13]:
import statsmodels.formula.api as smf

X = df["Close"].rename(columns={RISKY_ASSET: "asset",
                                    MARKET_BENCHMARK: "market"}) \
                   .resample("ME") \
                   .last() \
                   .pct_change() \
                   .dropna()

# define and fit the regression model
capm_model = smf.ols(formula="asset ~ market", data=X).fit()

# print results
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  asset   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.399
Method:                 Least Squares   F-statistic:                     39.50
Date:                Thu, 30 Oct 2025   Prob (F-statistic):           4.89e-08
Time:                        02:11:32   Log-Likelihood:                 70.999
No. Observations:                  59   AIC:                            -138.0
Df Residuals:                      57   BIC:                            -133.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0043      0.010      0.441      0.6